In [1]:
import sqlite3
from tabulate import tabulate


local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path


def add_new_user(user_id, conn):
    if not does_exist(user_id, conn):  # Check if the user exists before insertion
        try:
            cur = conn.cursor()
            query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
            cur.execute(query)
            conn.commit()
            print(f"User {str(user_id)} record added")
        except sqlite3.Error as e:
            print(f"Error adding user {user_id}: {e}")
    else:
        print(f"User {user_id} already exists")

def add_funds(user_id, amount, conn):
    if does_exist(user_id, conn):
        try:
            with conn:  # Use a with statement to ensure proper closure
                cur = conn.cursor()
                query = f"UPDATE savings SET balance = balance + {str(amount)} WHERE user_id = {str(user_id)}"
                cur.execute(query)
                print(f"Record for user: {str(user_id)} updated - {str(amount)} added")
        except sqlite3.Error as e:
            print(f"Error updating balance for user {user_id}: {e}")
    else:
        print("User does not exist")

# Function to create the database
def create_database(db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
    cur.execute(query)
    conn.commit()
    print("Database connected and savings table created if it didn't exist already")
    return conn

# Adjusted function to setup the database with initial data
def setup_database(conn):
    starting_balances = [44, 5, 42, 32, 25, 5, 26, 39, 45, 29]
    for id in range(1, 11):
        add_new_user(id, conn)
        add_funds(id, starting_balances[id - 1], conn)
    print("Test data added")

def does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT user_id FROM savings WHERE user_id = {user_id}"
    cur.execute(query)
    data = cur.fetchone()
    return data is not None

def get_data(conn):
    cur = conn.cursor()
    query = "SELECT * FROM savings"
    cur.execute(query)
    data = cur.fetchall()
    columns = [description[0] for description in cur.description]
    return data, columns

# Adjusted code to create the database in the D:/ path
local_db_path = "D:/savings.sqlite"  # Updated path
conn = create_database(local_db_path)  # Create the database

# Setup the database with initial data
setup_database(conn)  

# Fetch and display the data
data, columns = get_data(conn)
print(tabulate(data, headers=columns))

Database connected and savings table created if it didn't exist already
User 1 record added
Record for user: 1 updated - 44 added
User 2 record added
Record for user: 2 updated - 5 added
User 3 record added
Record for user: 3 updated - 42 added
User 4 record added
Record for user: 4 updated - 32 added
User 5 record added
Record for user: 5 updated - 25 added
User 6 record added
Record for user: 6 updated - 5 added
User 7 record added
Record for user: 7 updated - 26 added
User 8 record added
Record for user: 8 updated - 39 added
User 9 record added
Record for user: 9 updated - 45 added
User 10 record added
Record for user: 10 updated - 29 added
Test data added
  user_id    balance
---------  ---------
        1         44
        2          5
        3         42
        4         32
        5         25
        6          5
        7         26
        8         39
        9         45
       10         29


In [18]:
import sqlite3
from tabulate import tabulate


local_db_path = "D:/savings.sqlite"
conn = create_database("D:/savings.sqlite")

def get_user_balance(user_id, conn):
    if not isinstance(user_id, int):
        print("User ID should be an integer")
        return None

    if does_exist(user_id, conn):
        try:
            cur = conn.cursor()
            query = f"SELECT balance FROM savings WHERE user_id = {user_id}"
            cur.execute(query)
            balance = cur.fetchone()
            if balance:
                return balance[0]
            else:
                print(f"No balance found for user {user_id}")
                return None
        except sqlite3.Error as e:
            print(f"Error retrieving balance for user {user_id}: {e}")
            return None
    else:
        print(f"User {user_id} does not exist")
        return None

def get_all_users_data(conn):
    try:
        cur = conn.cursor()
        query = "SELECT * FROM savings"
        cur.execute(query)
        data = cur.fetchall()
        columns = [description[0] for description in cur.description]
        return data, columns
    except sqlite3.Error as e:
        print(f"Error retrieving all users' data: {e}")
        return None, None


def add_new_user(user_id, conn):
    if not isinstance(user_id, int):
        print("User ID should be an integer")
        return

    if does_exist(user_id, conn):
        print(f"User {user_id} already exists")
        return

    try:
        cur = conn.cursor()
        query = f"INSERT INTO savings(user_id,balance) VALUES({user_id},0)"
        cur.execute(query)
        conn.commit()
        print(f"User {user_id} record added")
    except sqlite3.Error as e:
        print(f"Error adding user {user_id}: {e}")


add_new_user(3, conn)        

def add_funds(user_id, amount, conn):
    if not (isinstance(user_id, int) and isinstance(amount, (int, float))):
        print("User ID should be an integer, and amount should be a number")
        return

    if does_exist(user_id, conn):
        try:
            with conn:
                cur = conn.cursor()
                query = f"UPDATE savings SET balance = balance + {amount} WHERE user_id = {user_id}"
                cur.execute(query)
                
                # Fetch the new balance
                cur.execute(f"SELECT balance FROM savings WHERE user_id = {user_id}")
                new_balance = cur.fetchone()[0]

                print(f"Record for user: {user_id} updated - {amount} added, new balance: {new_balance}")
                return new_balance
        except sqlite3.Error as e:
            print(f"Error updating balance for user {user_id}: {e}")
            return None
    else:
        print(f"User {user_id} does not exist. Cannot add funds.")
        return None

# Check if the user exists before adding funds
user_to_check = 15
if does_exist(user_to_check, conn):
    new_balance = add_funds(user_to_check, 15, conn)
    if new_balance is not None:
        print(f"New balance: {new_balance}")
else:
    print(f"User {user_to_check} does not exist. Cannot add funds.")

add_funds(4, 20, conn)

def remove_funds(user_id, amount, conn):
    if not (isinstance(user_id, int) and isinstance(amount, (int, float))):
        print("User ID should be an integer, and amount should be a number")
        return

    if does_exist(user_id, conn):
        current_balance = get_user_balance(user_id, conn)  # Fetch current balance
        if current_balance is None:
            print(f"Cannot retrieve balance for user {user_id}. Operation aborted.")
            return None

        new_balance = current_balance - amount
        if new_balance < 0:
            print(f"Cannot remove funds. Balance would go below zero for user {user_id}.")
            print(f"Current balance for user {user_id}: {current_balance}")
            return   # Show current balance instead of removing funds

        try:
            with conn:
                cur = conn.cursor()
                query = f"UPDATE savings SET balance = balance - {amount} WHERE user_id = {user_id}"
                cur.execute(query)
                
                # Fetch the new balance
                cur.execute(f"SELECT balance FROM savings WHERE user_id = {user_id}")
                new_balance = cur.fetchone()[0]

                print(f"Record for user: {user_id} updated - {amount} removed, new balance: {new_balance}")
                return 
        except sqlite3.Error as e:
            print(f"Error updating balance for user {user_id}: {e}")
            return None
    else:
        print(f"User {user_id} does not exist. Cannot remove funds.")
        return None
remove_funds(4, 15, conn)

def exit_app(conn):
    conn.close()
    print("Exiting app...")

     
def test_exit_app(conn):
    exit_app(conn)  # Close the connection
    try:
        conn.execute("SELECT * FROM savings")  # Attempt to execute a query
        print("Test failed: Connection is not properly closed.")
        return 
    except sqlite3.ProgrammingError:
        print("Test passed: Connection is properly closed.")
        return 
test_exit_app(conn)

Database connected and savings table created if it didn't exist already
User 3 already exists
User 15 does not exist. Cannot add funds.
Record for user: 4 updated - 20 added, new balance: 62.0
Record for user: 4 updated - 15 removed, new balance: 47.0
Exiting app...
Test passed: Connection is properly closed.
